# Spark Local & Spark Client mode 
- pyspark를 이용해 local mode로 스파크를 사용하는 샘플 코드  
- pyspark를 이용해 cluster mode 에 client오 연결해 스파크를 사용하는 샘플 코드 
Spark, Hadoop 를 별도로 설치하지 않고, pyspark 만을 설치한 상태에서 실행할 수 있다.  
단, Spark 자체가 jvm에서 작동하기 때문에 JDK 는 설치되어 있어야 한다.  
  
주피터에서 spark 실행 시, jvm 연동이 정상적으로 되게 하기 위해서는 jupyter 실행 전에 
미리 `JAVA_HOME`, `PATH`를 설정해 두는 것이 좋다.  
  
pyspark를 설치한 경우 `SPARK_HOME`은 `pyspark`를 실행할 수 있는 `bin/pyspark`(which pyspark로 확인)가 위치한 경로를 말한다.  

In [1]:
!pip install findspark
!pwd

/tf/notebooks/notebooks


In [2]:
# import libraries 
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
import findspark
import time 

In [3]:
# 전역변수 설정 
# local mode 
SPARK_LOCAL_MASTER = "local[3]"
# client mode 
SPARK_CLUSTER_MASTER = "spark://34.64.97.16:7077" 
SPARK_APP_NAME = "DATA-Preparation"
HOST_NAME = socket.gethostname()
PORT = 9999

# Define path
DATA_ROOT = f'{os.getcwd()}/data'
DATA_PATH = f'{DATA_ROOT}'

In [4]:
print(DATA_PATH)
print(HOST_NAME)

/tf/notebooks/notebooks/data
spark-client


In [5]:
# os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
# os.environ['PATH'] = '/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/'
os.environ['SPARK_HOME'] = '/usr/local'
# os.environ['HADOOP_HOME'] = '/hadoop-3.2.2'
# os.environ['hadoop.home.dir'] = '/hadoop-3.2.2/bin'
# os.environ['CLASSPATH'] = '$CLASSPATH:/hadoop-3.2.2/spark-3.2.0-bin-hadoop3.2.tar'
findspark.init()
print(os.getenv('JAVA_HOME'))

/usr/lib/jvm/java-8-openjdk-amd64/


In [6]:
print(f"JAVA_HOME : {os.getenv('JAVA_HOME')}")
print(f"PATH : {os.getenv('PATH')}")
os.getcwd()

JAVA_HOME : /usr/lib/jvm/java-8-openjdk-amd64/
PATH : /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin/


'/tf/notebooks/notebooks'

In [7]:
# Spark session 생성 메서드 
def init_remote_session():
    #SPARK_CLUSTER_MASTER = "spark://34.64.108.172:7077" 
    spark = SparkSession.builder.master(SPARK_CLUSTER_MASTER).appName(SPARK_APP_NAME).getOrCreate()
    return spark
    
# local mode 실행 시 메모리를 확장해야 하는 경우 있어서, conf에 memory 변경 추가  
def init_local_session():
    #SPARK_LOCAL_MASTER = "local[3]"
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config('spark.driver.host', HOST_NAME).getOrCreate()
    default_conf = spark.sparkContext._conf#.getAll()
    print(f'Old Conf : {default_conf.getAll()}')
    conf = spark.sparkContext._conf.setAll([
        ('spark.executor.instances', 1)
        #, ('spark.driver.memory', '12g'), ('spark.executor.memory', '8g'), ('spark.driver.maxResultSize', '8g')
        , ('spark.driver.allowMultipleContexts', 'true'), ('spark.sql.shuffle.partitions', 8)
        ##,('spark.memory.offHeap.enabled', True), ('spark.memory.offHeap.size', '8g')
    ])
    spark.sparkContext.stop()
    
    spark = SparkSession.builder.master(SPARK_LOCAL_MASTER).appName(SPARK_APP_NAME).config(conf=default_conf).getOrCreate()
    new_conf = spark.sparkContext._conf
    print(f'Updated Conf : {new_conf.getAll()}')
    return spark
    

In [46]:
# 스파크 생성 
#spark = init_local_session()
# 스파크 클러스터를 실행해 두었다고 가정
spark = init_remote_session()
sc = spark.sparkContext
sc

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


<SparkContext master=spark://34.64.97.16:7077 appName=DATA-Preparation>

In [47]:
# channel, day, seg, setop, 1000
FILE_FORMAT = 'parquet'
#FILE_FORMAT = 'csv'

# setop data n 개 만들기
def create_setops():
    setop_count = 10000
    setop_name = ['ST_A', 'ST_B', 'ST_C', 'ST_D', 'ST_E']
    setops = []
    for s in setop_name:
        for i in range(0, int(setop_count/len(setop_name))):
            setops.append(f'{s}_{i:03d}')
            
    print(setops[-10:])
    return setops

# channel, day, seg data 생성 
def create_others():
    # 20
    channels = ['KBS', 'MBC', 'SBS', 'JTBC', 'CBS' ,  'OCN', 'TVN', 'TVCH', 'BTN', 'EBS',  'Arirang', 'JTV', 'GAME-TV', 'HBC', 'BBC',  'CNN', 'CNBC', 'CCN', 'NHK', 'ABC'] 
    days = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
    hour_bands = ['00', '01', '10', '11', '23']
    segs = ['Agriculture', 'Game']
    rows = []
    row = []
    for c in channels:
        for d in days:
            for s in segs:
                row = [c, d, s]
                rows.append(row)
                
    print(rows[:5])
    return rows

def merge_to_inventory(setops, rows):
    inven_time = 10000 # 하드코딩 시간
    invens = []
    for r in rows:
        for s in setops:
            invens.append(r + [s, inven_time])
    print(f'Inventory Length : {len(invens):,}')
    return invens

def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType
    columns = [
        StructField("channel", StringType())
        , StructField("day", StringType())
        , StructField("seg", StringType())
        , StructField("setop", StringType())
        , StructField("remains", LongType())
    ]
    inven_schema = StructType(columns)
    return inven_schema

def save_inventory(invens, spark_session=spark, file_name=f'{DATA_PATH}/inven', sample_count=10000):
    inven_schema = define_schema()
    if (sample_count <= 0):
        # all data 
        rdd = spark_session.sparkContext.parallelize(invens)
    else:
        # sampling data
        rdd = spark_session.sparkContext.parallelize(invens[:sample_count])
    df = spark_session.createDataFrame(rdd, inven_schema)
    #df.write.save(path=file_name, format='csv', mode='append', sep=',')
    df.write.save(path=file_name, format=FILE_FORMAT, mode='append', sep=',')
        

In [48]:
%%time
setops = create_setops()

['ST_E_1990', 'ST_E_1991', 'ST_E_1992', 'ST_E_1993', 'ST_E_1994', 'ST_E_1995', 'ST_E_1996', 'ST_E_1997', 'ST_E_1998', 'ST_E_1999']
CPU times: user 4.3 ms, sys: 26 µs, total: 4.33 ms
Wall time: 4.27 ms


In [49]:
%%time
rows = create_others()

[['KBS', 'mon', 'Agriculture'], ['KBS', 'mon', 'Game'], ['KBS', 'tue', 'Agriculture'], ['KBS', 'tue', 'Game'], ['KBS', 'wed', 'Agriculture']]
CPU times: user 1.28 ms, sys: 152 µs, total: 1.43 ms
Wall time: 1.07 ms


In [50]:
%%time
invens = merge_to_inventory(setops, rows)

Inventory Length : 2,800,000
CPU times: user 1.87 s, sys: 212 ms, total: 2.08 s
Wall time: 2.08 s


In [52]:
%%time
# invens data 파일로 저장하기 
# gcp vm local : 12 s. csv - 90 MB, parq - 4.5 MB, 280만 rows 
SAMPLING_COUNT = int(1e7)
TABLE_NAME = f'{DATA_PATH}/inven'
# need file permission  
#TABLE_NAME = f'file:///home'

print(f'SAMPLING_COUNT : {SAMPLING_COUNT:,}')
print(f'TABLE_NAME : {TABLE_NAME}')
print(f'Inventory Length : {len(invens):,}')
      
save_inventory(invens, spark, TABLE_NAME, SAMPLING_COUNT)

SAMPLING_COUNT : 10,000,000
TABLE_NAME : file:///home
Inventory Length : 2,800,000


KeyboardInterrupt: 

In [43]:
%%time 
# 저장 결과 확인하기 
lines = spark.read.format(FILE_FORMAT).schema(define_schema()).option('path', TABLE_NAME).load()
data_count = lines.count()
print(f'DATA Count : {data_count:,}')
lines.show(5)

DATA Count : 2,800,000
+-------+---+----+--------+-------+
|channel|day| seg|   setop|remains|
+-------+---+----+--------+-------+
|    TVN|fri|Game|ST_B_864|  10000|
|    TVN|fri|Game|ST_B_865|  10000|
|    TVN|fri|Game|ST_B_866|  10000|
|    TVN|fri|Game|ST_B_867|  10000|
|    TVN|fri|Game|ST_B_868|  10000|
+-------+---+----+--------+-------+
only showing top 5 rows

CPU times: user 7.97 ms, sys: 465 µs, total: 8.44 ms
Wall time: 276 ms


In [44]:
%%time
# ch 별로 setop 번호 순서 5개씩 보여주기
# csv : 3.8 s, parq : 2.4s
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number 

windowPart = Window.partitionBy('channel').orderBy(col('setop').desc())
lines = spark.read.format(FILE_FORMAT).schema(define_schema()).option('path', TABLE_NAME).load()
df2 = lines.withColumn('row', row_number().over(windowPart)) .filter(col('row') <= 5)
df2.show(50)

+-------+---+-----------+--------+-------+---+
|channel|day|        seg|   setop|remains|row|
+-------+---+-----------+--------+-------+---+
|    ABC|mon|Agriculture|ST_E_999|  10000|  1|
|    ABC|mon|       Game|ST_E_999|  10000|  2|
|    ABC|tue|Agriculture|ST_E_999|  10000|  3|
|    ABC|tue|       Game|ST_E_999|  10000|  4|
|    ABC|wed|Agriculture|ST_E_999|  10000|  5|
|   CNBC|mon|Agriculture|ST_E_999|  10000|  1|
|   CNBC|mon|       Game|ST_E_999|  10000|  2|
|   CNBC|tue|Agriculture|ST_E_999|  10000|  3|
|   CNBC|tue|       Game|ST_E_999|  10000|  4|
|   CNBC|wed|Agriculture|ST_E_999|  10000|  5|
|GAME-TV|mon|Agriculture|ST_E_999|  10000|  1|
|GAME-TV|mon|       Game|ST_E_999|  10000|  2|
|GAME-TV|tue|Agriculture|ST_E_999|  10000|  3|
|GAME-TV|tue|       Game|ST_E_999|  10000|  4|
|GAME-TV|wed|Agriculture|ST_E_999|  10000|  5|
|    MBC|mon|Agriculture|ST_E_999|  10000|  1|
|    MBC|mon|       Game|ST_E_999|  10000|  2|
|    MBC|tue|Agriculture|ST_E_999|  10000|  3|
|    MBC|tue|

In [45]:
spark.stop()